Evaluate DeepTCR. Note that DeepTCR does _not_ provide a pre-trained model. This should be done under the `deeptcr` conda env on Python 3.8, which is NOT the same as our default environment (as it requires tensorflow instead of pytorch).

TF/CUDA (10.1) compatibility https://www.tensorflow.org/install/source#gpu

References:
- Manuscript: https://www.nature.com/articles/s41467-021-21879-w#data-availability
- Code: https://github.com/sidhomj/DeepTCR
- Tutorial for unsupervised training of DeepTCR https://github.com/sidhomj/DeepTCR/blob/master/tutorials/unsupervised/8%20-%20VAE%20Inference.ipynb

In [1]:
import os, sys
import collections
import logging
import itertools
from typing import *

import numpy as np 
import pandas as pd

SRC_DIR = os.path.join(os.path.dirname(os.getcwd()), "tcr")
assert os.path.isdir(SRC_DIR)
sys.path.append(SRC_DIR)
import canonical_models as models

RESULTS_DIR = os.path.join(
    os.path.dirname(SRC_DIR),
    "results", "lcmv_test_set",
)
assert os.path.isdir(RESULTS_DIR)

os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
from DeepTCR.DeepTCR import DeepTCR_U

DTCRU = DeepTCR_U('murine_antigens', device=3)
DTCRU

In [3]:
DTCRU.Get_Data(
    directory="/home/wukevin/downloads/DeepTCR/Data/Murine_Antigens",
    Load_Prev_Data=False, aggregate_by_aa=True, count_column=1,
    aa_column_beta=0,
)

Loading Data...
Embedding Sequences...
Data Loaded


In [4]:
DTCRU.Train_VAE(Load_Prev_Data=False)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
Epoch = 0, Iteration = 0 Total Loss: 1.11704: Recon Loss: 1.03774: Latent Loss: 0.07930: Sparsity Loss: 0.00000: Recon Accuracy: 0.03959
Epoch = 1, Iteration = 0 Total Loss: 1.10375: Recon Loss: 1.02754: Latent Loss: 0.07621: Sparsity Loss: 0.00000: Recon Accuracy: 0.06741
Epoch = 2, Iteration = 0 Total Loss: 1.09117: Recon Loss: 1.01822: Latent Loss: 0.07295: Sparsity Loss: 0.00000: Recon Accuracy: 0.10328
Epoch = 3, Iteration = 0 Total Loss: 1.07644: Recon Loss: 1.00874: Latent Loss: 0.06769: Sparsity Loss: 0.00000: Recon Accuracy: 0.14252
Epoch = 4, Iteration = 0 Total Loss: 1.05853: Recon Loss: 0.9

Epoch = 46, Iteration = 0 Total Loss: 0.61515: Recon Loss: 0.59174: Latent Loss: 0.02341: Sparsity Loss: 0.00000: Recon Accuracy: 0.45442
Epoch = 47, Iteration = 0 Total Loss: 0.61423: Recon Loss: 0.58984: Latent Loss: 0.02439: Sparsity Loss: 0.00000: Recon Accuracy: 0.45407
Epoch = 48, Iteration = 0 Total Loss: 0.61300: Recon Loss: 0.59176: Latent Loss: 0.02124: Sparsity Loss: 0.00000: Recon Accuracy: 0.46041
Epoch = 49, Iteration = 0 Total Loss: 0.61099: Recon Loss: 0.59002: Latent Loss: 0.02097: Sparsity Loss: 0.00000: Recon Accuracy: 0.45780
Epoch = 50, Iteration = 0 Total Loss: 0.60339: Recon Loss: 0.58093: Latent Loss: 0.02246: Sparsity Loss: 0.00000: Recon Accuracy: 0.46669
Epoch = 51, Iteration = 0 Total Loss: 0.60186: Recon Loss: 0.57973: Latent Loss: 0.02213: Sparsity Loss: 0.00000: Recon Accuracy: 0.46803
Epoch = 52, Iteration = 0 Total Loss: 0.60241: Recon Loss: 0.58200: Latent Loss: 0.02041: Sparsity Loss: 0.00000: Recon Accuracy: 0.46603
Epoch = 53, Iteration = 0 Total Lo

Epoch = 117, Iteration = 0 Total Loss: 0.47170: Recon Loss: 0.43576: Latent Loss: 0.03594: Sparsity Loss: 0.00000: Recon Accuracy: 0.60008
Epoch = 118, Iteration = 0 Total Loss: 0.46960: Recon Loss: 0.43336: Latent Loss: 0.03624: Sparsity Loss: 0.00000: Recon Accuracy: 0.60182
Epoch = 119, Iteration = 0 Total Loss: 0.46818: Recon Loss: 0.43241: Latent Loss: 0.03577: Sparsity Loss: 0.00000: Recon Accuracy: 0.60109
Epoch = 120, Iteration = 0 Total Loss: 0.46671: Recon Loss: 0.42949: Latent Loss: 0.03722: Sparsity Loss: 0.00000: Recon Accuracy: 0.60692
Epoch = 121, Iteration = 0 Total Loss: 0.46779: Recon Loss: 0.43066: Latent Loss: 0.03713: Sparsity Loss: 0.00000: Recon Accuracy: 0.60736
Epoch = 122, Iteration = 0 Total Loss: 0.46750: Recon Loss: 0.43093: Latent Loss: 0.03657: Sparsity Loss: 0.00000: Recon Accuracy: 0.60714
Epoch = 123, Iteration = 0 Total Loss: 0.46298: Recon Loss: 0.42536: Latent Loss: 0.03762: Sparsity Loss: 0.00000: Recon Accuracy: 0.61115
Epoch = 124, Iteration = 0 

Epoch = 176, Iteration = 0 Total Loss: 0.41538: Recon Loss: 0.36885: Latent Loss: 0.04653: Sparsity Loss: 0.00000: Recon Accuracy: 0.66899
Epoch = 177, Iteration = 0 Total Loss: 0.41088: Recon Loss: 0.36506: Latent Loss: 0.04582: Sparsity Loss: 0.00000: Recon Accuracy: 0.67287
Epoch = 178, Iteration = 0 Total Loss: 0.41078: Recon Loss: 0.36456: Latent Loss: 0.04621: Sparsity Loss: 0.00000: Recon Accuracy: 0.67317
Epoch = 179, Iteration = 0 Total Loss: 0.41077: Recon Loss: 0.36428: Latent Loss: 0.04649: Sparsity Loss: 0.00000: Recon Accuracy: 0.67309
Epoch = 180, Iteration = 0 Total Loss: 0.41204: Recon Loss: 0.36577: Latent Loss: 0.04627: Sparsity Loss: 0.00000: Recon Accuracy: 0.67230
Epoch = 181, Iteration = 0 Total Loss: 0.40978: Recon Loss: 0.36278: Latent Loss: 0.04700: Sparsity Loss: 0.00000: Recon Accuracy: 0.67652
Epoch = 182, Iteration = 0 Total Loss: 0.40691: Recon Loss: 0.35942: Latent Loss: 0.04750: Sparsity Loss: 0.00000: Recon Accuracy: 0.68046
Epoch = 183, Iteration = 0 

Epoch = 236, Iteration = 0 Total Loss: 0.36339: Recon Loss: 0.30977: Latent Loss: 0.05361: Sparsity Loss: 0.00000: Recon Accuracy: 0.72447
Epoch = 237, Iteration = 0 Total Loss: 0.36572: Recon Loss: 0.31258: Latent Loss: 0.05314: Sparsity Loss: 0.00000: Recon Accuracy: 0.72266
Epoch = 238, Iteration = 0 Total Loss: 0.36636: Recon Loss: 0.31331: Latent Loss: 0.05305: Sparsity Loss: 0.00000: Recon Accuracy: 0.72111
Epoch = 239, Iteration = 0 Total Loss: 0.36490: Recon Loss: 0.31264: Latent Loss: 0.05226: Sparsity Loss: 0.00000: Recon Accuracy: 0.72043
Epoch = 240, Iteration = 0 Total Loss: 0.36583: Recon Loss: 0.31355: Latent Loss: 0.05228: Sparsity Loss: 0.00000: Recon Accuracy: 0.72051
Epoch = 241, Iteration = 0 Total Loss: 0.36293: Recon Loss: 0.31006: Latent Loss: 0.05287: Sparsity Loss: 0.00000: Recon Accuracy: 0.72240
Epoch = 242, Iteration = 0 Total Loss: 0.36353: Recon Loss: 0.31058: Latent Loss: 0.05295: Sparsity Loss: 0.00000: Recon Accuracy: 0.72228
Epoch = 243, Iteration = 0 

...

Epoch = 366, Iteration = 0 Total Loss: 0.29715: Recon Loss: 0.22913: Latent Loss: 0.06802: Sparsity Loss: 0.00000: Recon Accuracy: 0.79265
Epoch = 367, Iteration = 0 Total Loss: 0.29482: Recon Loss: 0.22671: Latent Loss: 0.06811: Sparsity Loss: 0.00000: Recon Accuracy: 0.79476
Epoch = 368, Iteration = 0 Total Loss: 0.29633: Recon Loss: 0.22850: Latent Loss: 0.06782: Sparsity Loss: 0.00000: Recon Accuracy: 0.79226
Epoch = 369, Iteration = 0 Total Loss: 0.29497: Recon Loss: 0.22769: Latent Loss: 0.06728: Sparsity Loss: 0.00000: Recon Accuracy: 0.79415
Epoch = 370, Iteration = 0 Total Loss: 0.29656: Recon Loss: 0.23025: Latent Loss: 0.06631: Sparsity Loss: 0.00000: Recon Accuracy: 0.79201
Epoch = 371, Iteration = 0 Total Loss: 0.29719: Recon Loss: 0.23126: Latent Loss: 0.06594: Sparsity Loss: 0.00000: Recon Accuracy: 0.79194
Epoch = 372, Iteration = 0 Total Loss: 0.29596: Recon Loss: 0.23006: Latent Loss: 0.06590: Sparsity Loss: 0.00000: Recon Accuracy: 0.79180
Epoch = 373, Iteration = 0 

Epoch = 436, Iteration = 0 Total Loss: 0.27688: Recon Loss: 0.20565: Latent Loss: 0.07123: Sparsity Loss: 0.00000: Recon Accuracy: 0.81146
Epoch = 437, Iteration = 0 Total Loss: 0.27346: Recon Loss: 0.20211: Latent Loss: 0.07135: Sparsity Loss: 0.00000: Recon Accuracy: 0.81592
Epoch = 438, Iteration = 0 Total Loss: 0.27498: Recon Loss: 0.20353: Latent Loss: 0.07145: Sparsity Loss: 0.00000: Recon Accuracy: 0.81427
Epoch = 439, Iteration = 0 Total Loss: 0.27461: Recon Loss: 0.20329: Latent Loss: 0.07132: Sparsity Loss: 0.00000: Recon Accuracy: 0.81444
Epoch = 440, Iteration = 0 Total Loss: 0.27491: Recon Loss: 0.20352: Latent Loss: 0.07139: Sparsity Loss: 0.00000: Recon Accuracy: 0.81421
Epoch = 441, Iteration = 0 Total Loss: 0.27222: Recon Loss: 0.20085: Latent Loss: 0.07137: Sparsity Loss: 0.00000: Recon Accuracy: 0.81570
Epoch = 442, Iteration = 0 Total Loss: 0.27469: Recon Loss: 0.20281: Latent Loss: 0.07188: Sparsity Loss: 0.00000: Recon Accuracy: 0.81097
Epoch = 443, Iteration = 0 

Epoch = 495, Iteration = 0 Total Loss: 0.26160: Recon Loss: 0.18706: Latent Loss: 0.07454: Sparsity Loss: 0.00000: Recon Accuracy: 0.82815
Epoch = 496, Iteration = 0 Total Loss: 0.26247: Recon Loss: 0.18809: Latent Loss: 0.07438: Sparsity Loss: 0.00000: Recon Accuracy: 0.82720
Epoch = 497, Iteration = 0 Total Loss: 0.26116: Recon Loss: 0.18698: Latent Loss: 0.07418: Sparsity Loss: 0.00000: Recon Accuracy: 0.82824
Epoch = 498, Iteration = 0 Total Loss: 0.26459: Recon Loss: 0.19016: Latent Loss: 0.07443: Sparsity Loss: 0.00000: Recon Accuracy: 0.82526
Epoch = 499, Iteration = 0 Total Loss: 0.26184: Recon Loss: 0.18735: Latent Loss: 0.07450: Sparsity Loss: 0.00000: Recon Accuracy: 0.82593
Epoch = 500, Iteration = 0 Total Loss: 0.26256: Recon Loss: 0.18748: Latent Loss: 0.07508: Sparsity Loss: 0.00000: Recon Accuracy: 0.82692
Epoch = 501, Iteration = 0 Total Loss: 0.26255: Recon Loss: 0.18723: Latent Loss: 0.07532: Sparsity Loss: 0.00000: Recon Accuracy: 0.82915
Epoch = 502, Iteration = 0 

Epoch = 563, Iteration = 0 Total Loss: 0.24906: Recon Loss: 0.17183: Latent Loss: 0.07722: Sparsity Loss: 0.00000: Recon Accuracy: 0.84108
Epoch = 564, Iteration = 0 Total Loss: 0.24963: Recon Loss: 0.17248: Latent Loss: 0.07714: Sparsity Loss: 0.00000: Recon Accuracy: 0.83832
Epoch = 565, Iteration = 0 Total Loss: 0.25161: Recon Loss: 0.17423: Latent Loss: 0.07738: Sparsity Loss: 0.00000: Recon Accuracy: 0.83702
Epoch = 566, Iteration = 0 Total Loss: 0.25203: Recon Loss: 0.17455: Latent Loss: 0.07747: Sparsity Loss: 0.00000: Recon Accuracy: 0.83733
Epoch = 567, Iteration = 0 Total Loss: 0.24991: Recon Loss: 0.17279: Latent Loss: 0.07712: Sparsity Loss: 0.00000: Recon Accuracy: 0.84104
Epoch = 568, Iteration = 0 Total Loss: 0.24911: Recon Loss: 0.17222: Latent Loss: 0.07690: Sparsity Loss: 0.00000: Recon Accuracy: 0.83940
Epoch = 569, Iteration = 0 Total Loss: 0.24758: Recon Loss: 0.17091: Latent Loss: 0.07667: Sparsity Loss: 0.00000: Recon Accuracy: 0.84012
Epoch = 570, Iteration = 0 

INFO:tensorflow:murine_antigens/models/model_0/model.ckpt is not in all_model_checkpoint_paths. Manually adding it.
Training Done


## Load in LCMV data

In [5]:
###
### THIS IS COPIED FROM data_loaders.py DUE TO CROSS-ENVIRONMENT
### COMPATIBILITY ISSUES
###

def dedup(x: Iterable[Any]) -> List[Any]:
    """
    Dedup the given iterable, preserving order of occurrence
    >>> dedup([1, 2, 0, 1, 3, 2])
    [1, 2, 0, 3]
    >>> dedup(dedup([1, 2, 0, 1, 3, 2]))
    [1, 2, 0, 3]
    """
    # https://stackoverflow.com/questions/480214/how-do-you-remove-duplicates-from-a-list-whilst-preserving-order
    # Python 3.7 and above guarantee that dict is insertion ordered
    # sets do NOT do this, so list(set(x)) will lose order information
    return list(dict.fromkeys(x))

def load_lcmv_table(
    fname: str = "/home/wukevin/projects/tcr/tcr/data/lcmv_tetramer_tcr.txt",
    drop_na: bool = True,
    drop_unsorted: bool = True,
) -> pd.DataFrame:
    """Load the LCMV data table"""
    table = pd.read_csv(fname, delimiter="\t")
    if drop_na:
        table.dropna(axis=0, how="any", subset=["tetramer", "TRB", "TRA"], inplace=True)
    if drop_unsorted:
        drop_idx = table.index[table["tetramer"] == "Unsorted"]
        table.drop(index=drop_idx, inplace=True)

    # Take entires with multiple TRA or TRB sequences and split them, carrying over
    # all of the other metadata to each row
    dedup_rows = []
    for i, row in table.iterrows():
        for i, (tra, trb) in enumerate(
            itertools.product(row["TRA"].split(";"), row["TRB"].split(";"))
        ):
            new_row = row.copy(deep=True)
            new_row["TRA"] = tra
            new_row["TRB"] = trb
            dedup_rows.append(new_row)
    dedup_table = pd.DataFrame(dedup_rows, columns=table.columns)

    return dedup_table

def dedup_and_merge_labels(
    sequences: Sequence[str], labels: Sequence[str], sep: str = ","
) -> Tuple[List[str], List[str]]:
    """
    Remove duplicates in sequences and merge labels accordingly
    sep is the label separator, used to split and rejoin labels
    Return is sorted!

    >>> dedup_and_merge_labels(['a', 'b', 'a'], ['x', 'y', 'y'])
    (['a', 'b'], ['x,y', 'y'])
    >>> dedup_and_merge_labels(['a', 'b', 'a', 'a'], ['x', 'y', 'y,x', 'z'])
    (['a', 'b'], ['x,y,z', 'y'])
    >>> dedup_and_merge_labels(['a', 'b', 'd', 'c'], ['x', 'z', 'y', 'n'])
    (['a', 'b', 'c', 'd'], ['x', 'z', 'n', 'y'])
    """
    # unique returns the *sorted* unique elements of an array
    uniq_sequences, inverse_idx, uniq_seq_counts = np.unique(
        sequences, return_inverse=True, return_counts=True
    )
    uniq_labels, agg_count = [], 0
    # Walk through all unique sequences and fetch/merge corresponding labels
    for i, (seq, c) in enumerate(zip(uniq_sequences, uniq_seq_counts)):
        orig_idx = np.where(inverse_idx == i)[0]
        match_labels = dedup([labels[i] for i in orig_idx])
        if len(match_labels) == 1:
            uniq_labels.append(match_labels.pop())
        else:  # Aggregate labels
            aggregated_labels = dedup(
                list(
                    itertools.chain.from_iterable([l.split(sep) for l in match_labels])
                )
            )
            logging.debug(f"Merging {match_labels} -> {sep.join(aggregated_labels)}")
            agg_count += 1
            uniq_labels.append(sep.join(sorted(aggregated_labels)))
    assert len(uniq_sequences) == len(uniq_labels)
    logging.info(
        f"Deduped from {len(sequences)} -> {len(uniq_sequences)} merging {agg_count} labels"
    )
    return list(uniq_sequences), uniq_labels

def dedup_lcmv_table(
    lcmv_tab: pd.DataFrame,
    blacklist_label_combos: Sequence[str] = (
        "TetMid,TetNeg",
        "TetNeg,TetPos",
        "TetMid,TetNeg,TetPos",
    ),
) -> Tuple[List[Tuple[str, str]], List[str]]:
    """
    Return TRA and TRB pairs that are deduped and removes pairs with ambiguous labels

    This was implemented to centrally solve the issue where the LCMV table had duplicate and
    a few cases of ambiguous labels

    Returns two lists of equal length:
    - List of (TRA, TRB) pairs
    - List of corresponding labels (may be merged)
    """
    lcmv_ab = ["|".join(p) for p in zip(lcmv_tab["TRA"], lcmv_tab["TRB"])]
    lcmv_ab_dedup, lcmv_labels_dedup = dedup_and_merge_labels(
        lcmv_ab, list(lcmv_tab["tetramer"])
    )
    all_label_counter = collections.Counter(lcmv_labels_dedup)
    logging.info(f"Combined labels {all_label_counter.most_common()}")
    logging.info(f"Filtering out labels {blacklist_label_combos}")
    good_label_idx = [
        i for i, l in enumerate(lcmv_labels_dedup) if l not in blacklist_label_combos
    ]
    logging.info(f"Retaining {len(good_label_idx)} pairs with unambiguous labels")
    lcmv_ab_good = [lcmv_ab_dedup[i] for i in good_label_idx]
    lcmv_labels_good = [lcmv_labels_dedup[i] for i in good_label_idx]
    assert len(lcmv_ab_good) == len(lcmv_labels_good) == len(good_label_idx)
    label_counter = collections.Counter(lcmv_labels_good)
    logging.info(f"LCMV deduped labels: {label_counter.most_common()}")

    # Resplit into pairs
    lcmv_ab_good_split = [tuple(p.split("|")) for p in lcmv_ab_good]
    return lcmv_ab_good_split, lcmv_labels_good

lcmv = load_lcmv_table()

lcmv_dedup_tra_trb, lcmv_dedup_labels = dedup_lcmv_table(lcmv)
lcmv_dedup_tra, lcmv_dedup_trb = zip(*lcmv_dedup_tra_trb)
lcmv_dedup = pd.DataFrame(
    {
        "TRA": lcmv_dedup_tra,
        "TRB": lcmv_dedup_trb,
        "tetramer": lcmv_dedup_labels,
        "label": ["TetPos" in l or "TetMid" in l for l in lcmv_dedup_labels],  # Pos and mid are both positive labels
    }
)
lcmv_dedup

,TRA,TRB,tetramer,label
0,CAAAAAGNYKYVF,CASSLLGGSYEQYF,TetNeg,False
1,CAAAASNTNKVVF,CASSLGLGANTGQLYF,TetNeg,False
2,CAAAASSGSWQLIF,CASGPREANERLFF,TetNeg,False
3,CAAADNYAQGLTF,CASGEGPDYTF,TetNeg,False
4,CAAADNYAQGLTF,CASRDWGDEQYF,TetNeg,False
...,...,...,...,...
17697,CVVNTGKLTF,CASSYGNERLFF,TetNeg,False
17698,CVYSNNRIFF,CASSLWDRGDTQYF,TetNeg,False
17699,CWGSALGRLHF,CASSDQGANTEVFF,TetNeg,False
17700,CWGSALGRLHF,CASSSGLGAEQYF,TetNeg,False


In [6]:
lcmv_labels = np.array(lcmv_dedup['label'])
lcmv_labels

array([False, False, False, ..., False, False, False])

In [7]:
lcmv_features = DTCRU.Sequence_Inference(
    alpha_sequences=lcmv_dedup['TRA'],
    beta_sequences=lcmv_dedup['TRB'],
)
lcmv_features.shape

INFO:tensorflow:Restoring parameters from murine_antigens/models/model_0/model.ckpt


INFO:tensorflow:Restoring parameters from murine_antigens/models/model_0/model.ckpt


(17702, 256)

## Build a model

In [8]:
###
### Also copied
###
from typing import *

import numpy as np

def shuffle_indices_train_valid_test(
    idx: np.ndarray, valid: float = 0.15, test: float = 0.15, seed: int = 1234
) -> Tuple[np.ndarray]:
    """
    Given an array of indices, return indices partitioned into train, valid, and test indices
    >>> np.all(shuffle_indices_train_valid_test(np.arange(10000))[1] == shuffle_indices_train_valid_test(np.arange(10000))[1])
    True
    """
    np.random.seed(seed)  # For reproducible subsampling
    indices = np.copy(idx)  # Make a copy because shuffling occurs in place
    np.random.shuffle(indices)  # Shuffles inplace
    num_valid = int(round(len(indices) * valid)) if valid > 0 else 0
    num_test = int(round(len(indices) * test)) if test > 0 else 0
    num_train = len(indices) - num_valid - num_test
    assert num_train > 0 and num_valid >= 0 and num_test >= 0
    assert num_train + num_valid + num_test == len(
        indices
    ), f"Got mismatched counts: {num_train} + {num_valid} + {num_test} != {len(indices)}"

    indices_train = indices[:num_train]
    indices_valid = indices[num_train : num_train + num_valid]
    indices_test = indices[-num_test:]
    assert indices_train.size + indices_valid.size + indices_test.size == len(idx)

    return indices_train, indices_valid, indices_test


def split_arr(
    arr: np.ndarray, split: Literal["train", "valid", "test"], **kwargs
) -> np.ndarray:
    """
    Split the array into the given split. kwargs are fed to shuffle_indices_train_valid_test
    """
    split_to_idx = {"train": 0, "valid": 1, "test": 2}
    assert split in split_to_idx, f"Unrecognized split: {split}"
    indices = np.arange(arr.shape[0])
    keep_idx = shuffle_indices_train_valid_test(indices, **kwargs)[split_to_idx[split]]
    return arr[keep_idx]

lcmv_train = split_arr(lcmv_features, 'train')
lcmv_valid = split_arr(lcmv_features, 'valid')
lcmv_test = split_arr(lcmv_features, 'test')

In [9]:
lcmv_train_labels = split_arr(lcmv_labels, 'train')
lcmv_valid_labels = split_arr(lcmv_labels, 'valid')
lcmv_test_labels = split_arr(lcmv_labels, 'test')

In [10]:
np.mean(lcmv_test_labels)

0.1224105461393597

In [11]:
from sklearn.svm import SVC
from sklearn import metrics

factors = [1/n for n in [1, 2, 5, 10, 20, 50, 200]]
perf_df = pd.DataFrame(0, index=factors, columns=['AUROC', 'AUPRC'])

# Downsample the training data
for f in factors:
    downsample = int(round(lcmv_train.shape[0] * f))
    print(f"Downsampling: {f:.2f} {downsample}")
    svm = models.ModelOnPCA(SVC, probability=True, kernel='rbf')
    svm.fit(lcmv_train[:downsample], lcmv_train_labels[:downsample])
    
    train_preds = svm.predict_proba(lcmv_train)[:, 1]
    print(
        "Train",
        metrics.roc_auc_score(lcmv_train_labels[:downsample], train_preds[:downsample]),
        metrics.average_precision_score(lcmv_train_labels[:downsample], train_preds[:downsample]),
    )
    valid_preds = svm.predict_proba(lcmv_valid)[:, 1]
    print(
        "Valid",
        metrics.roc_auc_score(lcmv_valid_labels, valid_preds),
        metrics.average_precision_score(lcmv_valid_labels, valid_preds)
    )
    test_preds = svm.predict_proba(lcmv_test)[:, 1]
    test_auroc = metrics.roc_auc_score(lcmv_test_labels, test_preds)
    test_auprc = metrics.average_precision_score(lcmv_test_labels, test_preds)
    perf_df.loc[f] = (test_auroc, test_auprc)
    
    if np.isclose(f, 1.):
        np.savetxt(os.path.join(RESULTS_DIR, "deeptcr_lcmv_test_preds.txt"), test_preds)

perf_df

Downsampling: 1.00 12392
Train 0.8339431058838016 0.5576080300452536
Valid 0.7234457848607154 0.4371059070923571
Downsampling: 0.50 6196
Train 0.8596381221336673 0.5847632036332254
Valid 0.6791922631766383 0.3844725209313036
Downsampling: 0.20 2478
Train 0.8896685732976493 0.6462148767748798
Valid 0.6368516787353593 0.3333988908588738
Downsampling: 0.10 1239
Train 0.9426892483689338 0.7916565718909776
Valid 0.6241708956552706 0.3014797370134424
Downsampling: 0.05 620
Train 0.969628255342541 0.8873084590055046
Valid 0.6012886049778411 0.2758222412001479
Downsampling: 0.02 248
Train 0.9977956227365768 0.9788186849071929
Valid 0.5840177617521367 0.2847161485573306
Downsampling: 0.01 62
Train 1.0 1.0
Valid 0.5487576418566001 0.21968330269501343


,AUROC,AUPRC
1.000,0.744162,0.427299
0.500,0.707936,0.385975
0.200,0.662255,0.333094
0.100,0.609098,0.270248
0.050,0.578563,0.245355
0.020,0.575698,0.250932
0.005,0.533179,0.205940


In [12]:
perf_df.to_csv(os.path.join(RESULTS_DIR, "deeptcr_lcmv_test.csv"))